<a href="https://www.kaggle.com/code/bigswipp/mayo-image-segmentation?scriptVersionId=104199244" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
!pip install imutils

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd
import cv2 as cv
from imutils import paths
import skimage
from skimage.filters import sobel
from skimage import segmentation
from skimage.color import label2rgb
from skimage.color import rgb2hed, hed2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops, regionprops_table
from scipy import ndimage as ndi
import imageio.v2 as imageio
from sklearn.preprocessing import StandardScaler

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=b50df7d3b97558d80c5dc46ae5ebab875e2b0be63506ee073fedd627794c6463
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


!pip install imutils

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd
import cv2 as cv
from imutils import paths
import skimage
from skimage.filters import sobel
from skimage import segmentation
from skimage.color import label2rgb
from skimage.color import rgb2hed, hed2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops, regionprops_table
from scipy import ndimage as ndi
import imageio.v2 as imageio
from sklearn.preprocessing import StandardScaler

In [3]:
CE_folder_path = r'../input/mayo-compressed/MAYO/CE'
LAA_folder_path = r'../input/mayo-compressed/MAYO/LAA'

In [4]:
CE_images = sorted(list(paths.list_images(CE_folder_path)))
print('There are ' + str(len(CE_images)) + " images in the CE folder")
CE_images[:5]

There are 547 images in the CE folder


['../input/mayo-compressed/MAYO/CE/006388_0.png',
 '../input/mayo-compressed/MAYO/CE/008e5c_0.png',
 '../input/mayo-compressed/MAYO/CE/026c97_0.png',
 '../input/mayo-compressed/MAYO/CE/029c68_0.png',
 '../input/mayo-compressed/MAYO/CE/032f10_0.png']

In [5]:
LAA_images = sorted(list(paths.list_images(LAA_folder_path)))
print('There are ' + str(len(LAA_images)) + " images in the LAA folder")
LAA_images[:5]

There are 206 images in the LAA folder


['../input/mayo-compressed/MAYO/LAA/00c058_0.png',
 '../input/mayo-compressed/MAYO/LAA/01adc5_0.png',
 '../input/mayo-compressed/MAYO/LAA/028989_0.png',
 '../input/mayo-compressed/MAYO/LAA/03d1ec_0.png',
 '../input/mayo-compressed/MAYO/LAA/055f6a_0.png']

In [6]:
total_paths = CE_images + LAA_images
print(len(total_paths))
total_paths[:5]

753


['../input/mayo-compressed/MAYO/CE/006388_0.png',
 '../input/mayo-compressed/MAYO/CE/008e5c_0.png',
 '../input/mayo-compressed/MAYO/CE/026c97_0.png',
 '../input/mayo-compressed/MAYO/CE/029c68_0.png',
 '../input/mayo-compressed/MAYO/CE/032f10_0.png']

**Opening an Image**

In [7]:
def read_png(path):
    image = imageio.imread(path)
    filename = path.split('/')[-1].rstrip('.png')
    file_path = path 
    disease_class = path.split('/')[-2].rstrip('.png')
    #print("image_id: " + filename)
    return image, filename, disease_class, file_path

In [19]:
read_png(total_paths[0])

(Array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
  

**Converting the Image to Grayscale**

In [8]:
def convert_image_grayscale(image):
    gray_image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    return gray_image

In [20]:
#gray_img = convert_image_grayscale(image)

**Image Segmentation**

In [10]:
def segment_images(gray_img):
    elevation_map = sobel(gray_img) # creates image (array) that emphasizes edges
    markers = np.zeros_like(gray_img) # array of zeros in the same shape as the resized_gray_img array
    markers[gray_img >= gray_img.mean()] = 1
    markers[gray_img < gray_img.mean()] = 2
    segmented_img = segmentation.watershed(elevation_map, markers) # distinguishes objects in image 
    filled_segments = ndi.binary_fill_holes(segmented_img - 1) # method works with 0s and 1s (hence the -1, now 1 = area to keep). If 0 is eclosed by 1s -> 0 converted to 1
    labeled_segments, _ = ndi.label(filled_segments) # labels features in array, where non-zero values in input = features, and zero values = background.
    return labeled_segments

In [11]:
#labeled_segments = segment_images(gray_img)

**Object Coordinates**

In [12]:
def get_object_coordinates(labeled_segments):
    unique_filenames = []
    properties =['area','bbox','convex_area','bbox_area', 'major_axis_length', 'minor_axis_length', 'eccentricity']
    df = pd.DataFrame(regionprops_table(labeled_segments, properties=properties))
    standard_scaler = StandardScaler()
    scaled_area = standard_scaler.fit_transform(df.area.values.reshape(-1,1))
    df['scaled_area'] = scaled_area
    df.sort_values(by="scaled_area", ascending=False, inplace=True)
    
    for i in range(len(scaled_area)):
        unique_filenames.append(str(filename)+"_"+str(i+1))
    
    df['patch_name'] = unique_filenames
    
    df['class'] = [disease_class] * len(scaled_area)
    df['original_path'] = [file_path] * len(scaled_area)
    
    
    objects = df[df['scaled_area']>=.10] # tunable -> the lower it is, the more objects in the image will get boxed (original value = 0.75)
    
    simplified_df = objects.drop(['area', 'convex_area', 'bbox_area', 'major_axis_length', 'minor_axis_length', 'eccentricity', 'scaled_area'], axis=1)
    
    #display(simplified_df)
    object_coordinates = [(row['bbox-0'],row['bbox-1'],row['bbox-2'],row['bbox-3'] )for index, row in objects.iterrows()]
    
    simplified_df['object_coordinates'] = object_coordinates
    
    clean_df = simplified_df.drop(['bbox-0', 'bbox-1', 'bbox-2', 'bbox-3'], axis=1)
    
    display(clean_df)
    #return object_coordinates

In [13]:
#object_coordinates = get_object_coordinates(labeled_segments)

**Segment table**

In [15]:
def get_dataframe_from_image(path):
    image, file, disease_class, file_path = read_png(path)
    gray_img = convert_image_grayscale(image)
    labeled_segments = segment_images(gray_img)
    df = get_object_coordinates(labeled_segments)
    return df

#get_dataframe_from_image(CE_images[16])

**Segment table of every image**

In [17]:
def patches_df(paths_list):
    
    df_list = []
    
    for path in paths_list:
        
        image, filename, disease_class, file_path = read_png(path)
        gray_img = convert_image_grayscale(image)
        labeled_segments = segment_images(gray_img)
        object_coordinates = get_object_coordinates(labeled_segments) # df
        df_list.append(object_coordinates)
    
    combined_df = pd.concat(df_list)
    
    return combined_df

patches_df(total_paths)

NameError: name 'filename' is not defined